## Summary: Human-in-the-Loop Approvals and Edits in LangGraph Workflows  
Overview  
This demo introduces human-in-the-loop (HITL) techniques for LangGraph workflows, allowing users to approve, reject, or edit the state before certain actions (like tool calls) proceed. This control mechanism is vital for building safe and supervised AI systems.

Key Steps Covered  
1. Basic Workflow Structure  
a. State Definition
The custom state includes:
question: str
answer: str
MessagesState is inherited for managing message history.

class State(MessagesState):
  question: str
  answer: str  
b. Nodes
Entry Point:
Receives the user question and builds system and human messages.
Agent Node:
Binds the web search tool to the LLM.
Invokes the LLM with the current messages.
Updates the answer field.
def entry_point(state):
  ...
def agent(state):
  ...  
c. Router
If the latest message contains tool_calls, the workflow proceeds to tools.
Otherwise, it terminates.
def router(state):
  last_message = state["messages"][-1]
  return "tools" if last_message.tool_calls else END

In [ ]:
import os
from typing import Dict
from tavily import TavilyClient
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState
from langgraph.graph import START, END, StateGraph
from langgraph.prebuilt import ToolNode
from langgraph.graph.state import CompiledStateGraph
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from IPython.display import Image, display

In [ ]:
from dotenv import load_dotenv
load_dotenv()
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

In [ ]:
@tool
def web_search(question:str)->Dict:
    """
    Return top search results for a given search query
    """
    tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    response = tavily_client.search(question)
    return response

2. Introducing Breakpoints  
During graph compilation, interrupt_before=["tools"] is specified.

This pauses the workflow execution before tools are called.

Workflow visualization:

start → entry_point → agent → [breakpoint] tools → agent → end

graph = workflow.compile(interrupt_before=["tools"], checkpointer=memory)
Initial example:

Question: "What's the capital of Brazil?"

Workflow pauses after agent identifies a web search is needed.

In [ ]:
class State(MessagesState):
    question: str
    answer: str

In [ ]:
llm_with_tools = llm.bind_tools([web_search])

In [ ]:
def entry_point(state: State):
    question = state["question"]
    system_message = SystemMessage("You conduct web search to respond to user's questions")
    human_message = HumanMessage(question)
    messages = [system_message, human_message]
    return {"messages": messages}

In [ ]:
def agent(state: State):
    messages = state["messages"]
    ai_message = llm_with_tools.invoke(messages)
    return {"messages": ai_message, "answer": ai_message.content}

In [ ]:
def router(state: MessagesState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"

    return END

In [ ]:
workflow = StateGraph(State)
workflow.add_node("entry_point", entry_point)
workflow.add_node("agent", agent)
workflow.add_node("tools", ToolNode([web_search]))

workflow.add_edge(START, "entry_point")
workflow.add_edge("entry_point", "agent")
workflow.add_conditional_edges(
    source="agent", 
    path=router, 
    path_map=["tools", END]
)
workflow.add_edge("tools", "agent")

In [ ]:
memory = MemorySaver()
graph = workflow.compile(
    interrupt_before=["tools"], 
    checkpointer=memory
)

In [ ]:
display(
    Image(
        graph.get_graph().draw_mermaid_png()
    )
)

In [ ]:
input_question = {"question": "what's the capital of Brazil?"}
config = {"configurable": {"thread_id": 1}}

In [ ]:
for event in graph.stream(input=input_question, config=config, stream_mode="values"):
    if not event['messages']:
        continue
    event['messages'][-1].pretty_print()

In [ ]:
state = graph.get_state(config=config)

In [ ]:
state

In [ ]:
state.next

In [ ]:
for event in graph.stream(input=None, config=config, stream_mode="values"):
    if not event['messages']:
        continue
    event['messages'][-1].pretty_print()

## 3. Human Approval Flow  
a. Basic Approval  
A function human_in_the_loop_run streams the workflow.

After seeing the agent’s intent (e.g., to call a tool), the human is prompted:

YES: Continue execution.

NO: Abort workflow.


human_input = input("Do you approve the tool calling? (YES or NO): ")
Example:

Approving → Continues and retrieves the correct answer.

Rejecting → Prints "Workflow aborted by a human."

b. More Interactive Control (Human Revision)  
Workflow is modified to interrupt before the agent instead of tools.

If the user says NO to the initial question:

They are prompted to edit the question.

A system message ("Workflow edited by a human") and a new human message are inserted.


human_input = input("So what should be the question? ")
The graph state is updated with the revised input.

Execution then resumes from the updated state.

Example:

Original: "What’s the capital of Brazil?"

Edit to: "What’s the capital of Canada?"

Output: "The capital of Canada is Ottawa."

In [ ]:
def human_in_the_loop_run(graph:CompiledStateGraph, question:str, thread_id:int):
    input_question = {"question": question}
    config = {"configurable": {"thread_id": thread_id}}
    for event in graph.stream(input=input_question, config=config, stream_mode="values"):
        if not event['messages']:
            continue
        event['messages'][-1].pretty_print()
    
    human_input = input("Do you approve the tool calling? (YES or NO): ")
    if human_input.lower() == "yes":
        for event in graph.stream(input=None, config=config, stream_mode="values"):
            if not event['messages']:
                continue
            event['messages'][-1].pretty_print()
    
    else:
        SystemMessage("Workflow aborted by a human").pretty_print()

In [ ]:
human_in_the_loop_run(
    graph=graph,
    question="what's the capital of Brazil?",
    thread_id=2,
)

In [ ]:
human_in_the_loop_run(
    graph=graph,
    question="what's the capital of Mexico?",
    thread_id=3,
)

## 4. Second Human Step After Tool Messages  
After a tool call, the human again can:

Review.

Continue execution.

Potentially enhance logic to approve post-tool actions.

In [ ]:
memory = MemorySaver()
graph = workflow.compile(
    interrupt_before=["agent"], 
    checkpointer=memory
)

In [ ]:
display(
    Image(
        graph.get_graph().draw_mermaid_png()
    )
)

In [ ]:
def human_in_the_loop_run(graph:CompiledStateGraph, question:str, thread_id:int):
    input_question = {"question": question}
    config = {"configurable": {"thread_id": thread_id}}
    for event in graph.stream(input=input_question, config=config, stream_mode="values"):
        if not event['messages']:
            continue
        event['messages'][-1].pretty_print()

    human_input = input("Do you want to proceed with this question? (YES or NO): ")
    
    if human_input.lower() == "no":
        human_input = input("So what should be the question? ")
        human_message = HumanMessage(content=human_input)
        system_message = SystemMessage("Workflow edited by a human")
        system_message.pretty_print()
        graph.update_state(
            config=config,
            values={
                "messages": [
                    system_message,
                    human_message,
                ]
            },
        )

        for event in graph.stream(input=None, config=config, stream_mode="values"):
            if not event['messages']:
                continue
            event['messages'][-1].pretty_print()
    
    for event in graph.stream(input=None, config=config, stream_mode="values"):
        if not event['messages']:
            continue
        event['messages'][-1].pretty_print()


In [ ]:
human_in_the_loop_run(
    graph=graph,
    question="what's the capital of Brazil?",
    thread_id=4,
)

In [ ]:
human_in_the_loop_run(
    graph=graph,
    question="what's the capital of Brazil?",
    thread_id=5,
)

## 5. Key Concepts Highlighted  
Breakpoints allow controlled interruptions during workflows.

Human approvals ensure critical actions (like external API calls) can be supervised.

Human edits allow correction of user input before significant processing happens.

Thread IDs manage session persistence, enabling inspection and modification across sessions.

## 6. Conclusion  
Human-in-the-loop mechanisms enhance AI system transparency and safety.

LangGraph’s breakpoint and checkpoint features make integrating HITL workflows straightforward.

A next enhancement would be extending approval loops after tool messages, enabling full-cycle human supervision.